# Music clustering

## Prerequisites

In [6]:
%config IPCompleter.greedy=True
import sys
print(sys.version)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


Save the current notebook session

In [25]:
import dill
dill.dump_session('data/notebook_session_env.dump')

Load a previous notebook session

In [1]:
import dill
dill.load_session('data/notebook_session_env.dump')

## Loading the data

In [5]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd

In [6]:
req = urlopen("https://os.unil.cloud.switch.ch/fma/fma_metadata.zip").read()
archive = ZipFile(BytesIO(req))

In [9]:
features = pd.read_csv(archive.open("fma_metadata/features.csv"))
features.head()

/home/tsb/.conda/envs/tensor/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,statistics,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
1,number,01,02,03,04,05,06,07,08,09,...,04,05,06,01,01,01,01,01,01,01
2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
4,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02


## Data normalization & reduction

Removing the redundant entries

In [11]:
features = features.drop([0, 1, 2])

In [2]:
features = features.reset_index()

In [3]:
features = features.drop(columns=['index'])

In [4]:
features.head()

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
1,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
2,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
3,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,7.4357867241e-02,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02
4,20,-1.9383698702e-01,-1.9852678478e-01,2.0154602826e-01,2.5855624676e-01,7.7520370483e-01,8.4794059396e-02,-2.8929358721e-01,-8.1641042233e-01,4.3850939721e-02,...,9.5002755523e-02,2.2492416203e-02,2.1355332807e-02,1.6669036865e+01,4.6972656250e-01,4.7224905342e-02,4.0039062500e-02,9.7656250000e-04,3.1898307800e+00,3.0992921442e-02



Removing entries with missing values

In [5]:
print("Entries before clean-up: {}".format(features["feature"].count()))
features = features.dropna()
print("Entries after clean-up: {}".format(features["feature"].count()))

Entries before clean-up: 106574
Entries after clean-up: 106574


Converting all the values to floats

In [8]:
for i in features:
    features[i] = features[i].astype("float")
features.dtypes

feature          float64
chroma_cens      float64
chroma_cens.1    float64
chroma_cens.2    float64
chroma_cens.3    float64
                  ...   
zcr.2            float64
zcr.3            float64
zcr.4            float64
zcr.5            float64
zcr.6            float64
Length: 519, dtype: object

Normalizing all the values

In [9]:
def normalize(x, x_min, x_max):
    return (x - x_min)/(x_max - x_min)

In [10]:
features_min_max = []

for feature in features:
    features_min_max.append((features[feature].min(), features[feature].max()))

features_min_max

[(2.0, 155320.0),
 (-1.9558056593000002, 8843.3671875),
 (-1.9366068840000001, 2864.0751953000004),
 (-1.9674503803, 1777.6159667999998),
 (-3.0, 4257.0458984),
 (-1.9212056398, 3188.0344238000002),
 (-1.9237399101, 2809.7792969),
 (-1.9353823662, 4281.0908203),
 (-3.0, 3869.4821776999997),
 (-3.0, 1217.2094727),
 (-3.0, 1772.9182129),
 (-3.0, 3887.3796386999993),
 (-3.0, 4995.784668),
 (0.021006945521, 1.0),
 (0.051881998777, 0.7698003649699999),
 (0.054485078901, 0.75904983282),
 (0.0, 0.7698003649699999),
 (0.038673587143, 0.7698003649699999),
 (0.021914415061, 0.7698003649699999),
 (0.020284691825999998, 0.7698003649699999),
 (0.0, 0.7698003649699999),
 (0.0, 0.7698003649699999),
 (0.0, 0.7698003649699999),
 (0.0, 0.84798967838),
 (0.0, 1.0),
 (6.4476616899e-06, 0.83884578943),
 (7.4612296885e-05, 0.69208371639),
 (5.493977369e-05, 0.6740944385500001),
 (0.0, 0.73191124201),
 (2.3917373255e-05, 0.67062103748),
 (2.4554037735999996e-05, 0.65820920467),
 (4.9362395656999994e-05, 0.67

In [11]:
i = 0
for key, value in features.iteritems():
    features[key] = normalize(value, features_min_max[i][0], features_min_max[i][1])
    i += 1
    
features[feature].max(), features[feature].min(), features.head()

(1.0,
 0.0,
     feature  chroma_cens  chroma_cens.1  chroma_cens.2  chroma_cens.3  \
 0  0.000000     0.001033       0.002501       0.001246       0.001021   
 1  0.000006     0.000435       0.000941       0.001300       0.001243   
 2  0.000019     0.000281       0.000649       0.000948       0.000865   
 3  0.000052     0.000640       0.000574       0.002340       0.000649   
 4  0.000116     0.000199       0.000606       0.001219       0.000765   
 
    chroma_cens.4  chroma_cens.5  chroma_cens.6  chroma_cens.7  chroma_cens.8  \
 0       0.001067       0.000873       0.000798       0.001470       0.003169   
 1       0.001121       0.000708       0.000771       0.001047       0.002547   
 2       0.001210       0.000997       0.000236       0.000535       0.003005   
 3       0.001031       0.001039       0.000865       0.001189       0.002886   
 4       0.000845       0.000714       0.000384       0.000564       0.002495   
 
    ...  tonnetz.39  tonnetz.40  tonnetz.41       zcr 

Converting the dataset from Pandas to a NumPy array

In [12]:
data = features.to_numpy()
data

array([[0.00000000e+00, 1.03291404e-03, 2.50065819e-03, ...,
        0.00000000e+00, 1.89929477e-01, 1.43937014e-01],
       [6.43840379e-06, 4.34666886e-04, 9.41079927e-04, ...,
        0.00000000e+00, 1.85951899e-01, 1.62532003e-01],
       [1.93152114e-05, 2.80754998e-04, 6.48619997e-04, ...,
        0.00000000e+00, 1.91032000e-01, 1.04804513e-01],
       ...,
       [9.99987123e-01, 2.83571226e-04, 6.37160204e-04, ...,
        1.30353818e-02, 1.76484607e-01, 4.12105258e-02],
       [9.99993562e-01, 2.01112481e-04, 7.41035027e-04, ...,
        1.86219739e-02, 1.87039344e-01, 4.64692745e-02],
       [1.00000000e+00, 2.76470453e-04, 1.32554515e-03, ...,
        1.67597765e-02, 2.17615764e-01, 3.22658994e-01]])

## Train/Test data split

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(data, test_size=1/5, random_state=0)
'Train entries: {} / Test entries: {}''.format(len(x_train), len(x_test))

'Train entries: 85259 / Test entries: 21315'

## PCA data reduction

Figuring out the optimal amount of PCA dimensions in order to retain 97.5% of the data

In [18]:
from sklearn.decomposition import PCA

pca = PCA(.975)
pca.fit(x_train)
'PCA Dimensions: {}'.format(pca.n_components_)

'PCA Dimensions: 162'

Applying PCA to the datasets

In [23]:
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)
len(x_train[0]), len(x_test[0])

(162, 162)